## Libraries

In [20]:
!pip install peft
!pip install bitsandbytes

  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 3.5 MB/s eta 0:00:0000:0100:01m
Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl (23.1 MB)


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model

/Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Device

In [2]:
device = torch.device("mps")  # Set MPS as the device

## LoRa Fine-Tuning

In [3]:
lora_config = LoraConfig(
    r=16,  
    lora_alpha=32,  
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05,  
    bias="none"  
)

## Model and Tokenizer

In [4]:
!pip install bitsandbytes==0.40.0


In [5]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.40.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages
Requires: 
Required-by: 


In [6]:
# Load the model and tokenizer
model_name = "arcee-ai/Llama-3.1-SuperNova-Lite"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto")

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.73s/it]
Some parameters are on the meta device because they were offloaded to the disk.


## Applying and Verifying LoRa

In [7]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  

/Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/mehran/CodeSpaces/Testing/Table_Answering/env/lib/python3.12/site-pack

# Dataset

In [12]:
from datasets import load_dataset

train_dataset = load_dataset("../data/queries_final.csv")
# val_dataset = load_dataset("path_to_val_file.csv")

FileNotFoundError: Couldn't find any data file at /Users/mehran/CodeSpaces/Testing/Table_Answering/data/queries_final.csv.

## Preprocessing

In [ ]:
def prepocess_text(examples):
    return tokenizer(examples['text'], padding='max_length',  truncation=True, max_length=512)

In [ ]:
train_tokenized = train_dataset.map(prepocess_text, batched=True)
val_tokenized = val_dataset.map(prepocess_text, batched=True)

## Data Collator

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Training Arguments

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    device="mps",
    fp16=True,
    report_to="none"
)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (753927232.py, line 13)

## Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,   
    tokenizer=tokenizer,
    data_collator=data_collator
)

## Training The Model

In [ ]:
trainer.train()

# Manual Training Loop

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-4).to(device)
model.train()

for epoch in range(3):  # Number of epochs
    for batch in train_dataloader:
        inputs = tokenizer(batch["text"], return_tensors="pt", padding=True, truncation=True).to(device)
        labels = inputs["input_ids"].to(device)
        outputs = model(**inputs, labels=labels).to(device)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


# Saving model and Tokenizer    

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


# Inference

TODO: Add Testing and Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("mps")  # Use MPS

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model").to(device)

# Generate text
input_text = "Your prompt here"
inputs = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
